# 报告生成agent

1、根据输入的请求和文本，生成 任务规划+行文分析  
2、根据 任务规划，并发调用外部工具，得到 结果列表  
3、根据 行文分析，生成 章节目录  
4、根据 结果列表+章节目录，生成报告内容  

In [1]:
# 库导入
from langchain_deepseek import ChatDeepSeek
from typing import Optional

from pydantic import BaseModel, Field

import asyncio

import os
from dotenv import load_dotenv


In [2]:
load_dotenv()
model_name = os.getenv("DEEPSEEK_MODEL", "deepseek-chat")
api_key = os.getenv("DEEPSEEK_API_KEY")

if not api_key:
    raise ValueError("API_KEY environment variable is required")

llm = ChatDeepSeek(
    model=model_name,
    temperature=1,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=api_key,
)



# 测试文本

In [20]:
# 请求和文本输入
reguest = "请根据下面的文章生成一份报告"
text = """
纽约天气
华盛顿天气
纽约人口
"""

# 任务规划
这一部分可以考虑使用MCP实现，更具有通用性
MCP的测试代码已经跑通，具体逻辑暂未合并到本文件中

In [ ]:

# 已有的function列表
functionlist = ["get_weather", "get_population"]

# funtioncalling规划格式定义
class callinglist(BaseModel):
    '''用于function calling的规划格式'''
    function: str = Field(description="The name of the function to call")
    arguments: str = Field(description="The arguments to pass to the function")




In [ ]:
# 大模型分析 request 和 text ，以及一份现有的functionlist生成一份格式化的function calling规划，
structured_llm = llm.with_structured_output(callinglist)
plan = structured_llm.invoke(reguest + text + str(functionlist))


In [ ]:
# 解析plan，合并结果

SearchResults = []
for item in plan:
    pass

# SearchResults 转换为字符串打印，作为搜索结果报告
print("搜索结果报告：")
for result in SearchResults:
    print(result)

# 行文分析

In [12]:
# 大模型根据初始信息生成outline
outline = llm.invoke("根据下面的信息，生成一份用于指导生成报告的outline"+reguest + text)
outline


AIMessage(content='### 报告大纲：纽约与华盛顿的天气及人口概况  \n\n#### **一、引言**  \n1. 报告目的：比较纽约与华盛顿的天气特征及人口概况  \n2. 数据来源：提供的天气与人口信息  \n\n#### **二、纽约概况**  \n1. **天气特征**  \n   - 气候类型（如温带大陆性气候）  \n   - 四季温度范围（夏季/冬季平均气温）  \n   - 降水与极端天气（如降雪、暴雨频率）  \n2. **人口数据**  \n   - 总人口数量（最新统计）  \n   - 人口密度与分布（如市区 vs. 郊区）  \n   - 人口趋势（增长/减少原因）  \n\n#### **三、华盛顿概况**  \n1. **天气特征**  \n   - 气候类型（如湿润亚热带气候）  \n   - 季节性差异（夏季湿度、冬季温和程度）  \n   - 特殊天气现象（如飓风影响）  \n2. **人口数据**  \n   - 总人口数量（与纽约对比）  \n   - 人口结构（如政治从业者比例）  \n   - 流动人口特点（旅游或公务需求）  \n\n#### **四、纽约与华盛顿的对比分析**  \n1. **天气差异**  \n   - 温度、降水、极端天气频率对比  \n2. **人口差异**  \n   - 规模、密度、职业分布对比  \n3. **潜在影响因素**  \n   - 地理（沿海 vs. 内陆）、经济（商业 vs. 政治中心）  \n\n#### **五、结论与建议**  \n1. 总结两城市的主要差异  \n2. 对研究或决策的启示（如搬迁、旅游规划）  \n\n#### **六、附录（可选）**  \n- 数据表格（天气与人口具体数值）  \n- 参考文献（如有外部数据来源）  \n\n---  \n**备注**：可根据实际数据补充细节（如具体温度数值、人口年份等）。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 421, 'prompt_tokens': 32, 'total_tokens': 453, 'completion_tokens_deta

# 目录生成

In [ ]:
# 目录定义
class Ariticle(BaseModel):
    title: str = Field(description="文章标题")
    chapters: Optional[list["Chapter"]] = Field(default_factory=list,description="章节列表")

class Chapter(BaseModel):
    title: str = Field(description="章节标题")
    sections: Optional[list["Section"]] = Field(default_factory=list,description="一级子目录列表")
    
class Section(BaseModel):
    title: str = Field(description="一级子目录标题")
    subsections: Optional[list["Subsection"]] = Field(default_factory=list,description="二级子目录列表")
    
class Subsection(BaseModel):
    title: str = Field(description="二级子目录标题")
    contents: Optional[list["Content"]] = Field(default_factory=list,description="正文内容")

# 正文定义
class Content(BaseModel):
    text: str = Field(description="正文内容，markdown格式")
# 大模型根据outline生成目录

Articlellm = llm.with_structured_output(Ariticle)
catalog = Articlellm.invoke ("根据outline生成一篇报告目录"+outline.content)
catalog

Ariticle(title='纽约与华盛顿的天气及人口概况', chapters=[Chapter(title='一、引言', sections=[Section(title='报告目的：比较纽约与华盛顿的天气特征及人口概况', subsections=[]), Section(title='数据来源：提供的天气与人口信息', subsections=[])]), Chapter(title='二、纽约概况', sections=[Section(title='天气特征', subsections=[Subsection(title='气候类型（如温带大陆性气候）', contents=[]), Subsection(title='四季温度范围（夏季/冬季平均气温）', contents=[]), Subsection(title='降水与极端天气（如降雪、暴雨频率）', contents=[])]), Section(title='人口数据', subsections=[Subsection(title='总人口数量（最新统计）', contents=[]), Subsection(title='人口密度与分布（如市区 vs. 郊区）', contents=[]), Subsection(title='人口趋势（增长/减少原因）', contents=[])])]), Chapter(title='三、华盛顿概况', sections=[Section(title='天气特征', subsections=[Subsection(title='气候类型（如湿润亚热带气候）', contents=[]), Subsection(title='季节性差异（夏季湿度、冬季温和程度）', contents=[]), Subsection(title='特殊天气现象（如飓风影响）', contents=[])]), Section(title='人口数据', subsections=[Subsection(title='总人口数量（与纽约对比）', contents=[]), Subsection(title='人口结构（如政治从业者比例）', contents=[]), Subsection(title='流动人口特点（旅游或公务需求）', contents=[])])]

# 正文生成

In [ ]:
# 异步请求正文内容函数定义
Contentllm = llm.with_structured_output(Content)
async def fetch_content(title):
    print(f"开始生成: {title}")
    content = await Contentllm.ainvoke(title + "的正文内容")
    print(f"完成生成: {content}")
    return content

# 收集所有异步任务
tasks = []
for chapter in catalog.chapters:
    for section in chapter.sections:
        for subsection in section.subsections:
            tasks.append(fetch_content(subsection.title))

# 执行所有异步任务（不显示进度条）
article_contents = await asyncio.gather(*tasks)
    
# 将正文内容添加到目录中
for chapter, content in zip(catalog.chapters, article_contents):
    for section in chapter.sections:
        for subsection in section.subsections:
            subsection.contents.append(Content(text=content.text))
            

catalog



开始生成: 气候类型（如温带大陆性气候）
开始生成: 四季温度范围（夏季/冬季平均气温）
开始生成: 降水与极端天气（如降雪、暴雨频率）
开始生成: 总人口数量（最新统计）
开始生成: 人口密度与分布（如市区 vs. 郊区）
开始生成: 人口趋势（增长/减少原因）
开始生成: 气候类型（如湿润亚热带气候）
开始生成: 季节性差异（夏季湿度、冬季温和程度）
开始生成: 特殊天气现象（如飓风影响）
开始生成: 总人口数量（与纽约对比）
开始生成: 人口结构（如政治从业者比例）
开始生成: 流动人口特点（旅游或公务需求）
开始生成: 温度、降水、极端天气频率对比
开始生成: 规模、密度、职业分布对比
开始生成: 地理（沿海 vs. 内陆）、经济（商业 vs. 政治中心）
完成生成: text='根据最新统计数据，全球总人口数量约为80亿人。这一数据基于联合国和其他国际组织的综合统计，涵盖了各个国家和地区的人口普查结果。人口增长趋势显示，尽管增长率有所放缓，但全球人口仍在持续增加。亚洲和非洲是人口增长最快的地区，而欧洲和北美洲的人口增长相对较慢。\n\n需要注意的是，人口统计数据会因统计方法和时间点的不同而有所差异，因此具体数字可能会有所调整。如需更详细或特定地区的数据，建议参考官方统计机构的最新报告。'
完成生成: text='四季的温度范围因地理位置和气候类型的不同而有所差异。以下是夏季和冬季的平均气温范围的一般描述：\n\n### 夏季平均气温\n- **温带地区**：夏季平均气温通常在20°C至30°C之间，部分地区可能更高。\n- **热带地区**：夏季平均气温较高，通常在25°C至35°C之间，甚至更高。\n- **寒带地区**：夏季平均气温较低，通常在10°C至20°C之间。\n\n### 冬季平均气温\n- **温带地区**：冬季平均气温通常在-10°C至10°C之间，具体取决于纬度。\n- **热带地区**：冬季平均气温较为温和，通常在15°C至25°C之间。\n- **寒带地区**：冬季平均气温极低，通常在-30°C至-10°C之间，甚至更低。\n\n需要注意的是，这些范围仅为一般参考，具体温度会因地理位置、海拔高度和气候条件的不同而有所变化。'
完成生成: text='总人口数量是衡量一个城市或地区规模的重要指标之一。纽约作为美国最大的城市之一，其人口

Ariticle(title='纽约与华盛顿的天气及人口概况', chapters=[Chapter(title='一、引言', sections=[Section(title='报告目的：比较纽约与华盛顿的天气特征及人口概况', subsections=[]), Section(title='数据来源：提供的天气与人口信息', subsections=[])]), Chapter(title='二、纽约概况', sections=[Section(title='天气特征', subsections=[Subsection(title='气候类型（如温带大陆性气候）', contents=[Content(text='四季温度范围因地理位置和气候类型的不同而有所差异。以下是夏季和冬季的平均气温范围的一般描述：\n\n1. **夏季**：\n   - 平均气温通常在 **20°C 至 35°C** 之间，具体取决于地区。\n   - 热带地区夏季气温可能更高，甚至超过 **40°C**。\n   - 温带地区夏季较为温和，平均气温在 **20°C 至 30°C** 之间。\n\n2. **冬季**：\n   - 平均气温通常在 **-10°C 至 10°C** 之间，具体取决于地区。\n   - 寒带或高纬度地区冬季气温可能低至 **-30°C 或更低**。\n   - 亚热带地区冬季较为温暖，平均气温在 **10°C 至 20°C** 之间。\n\n需要注意的是，这些数据仅为一般参考，实际气温会因地形、海洋影响和气候变化等因素而有所不同。'), Content(text='四季的温度范围因地理位置和气候类型的不同而有所差异。以下是夏季和冬季的平均气温范围的一般描述：\n\n### 夏季平均气温\n- **温带地区**：夏季平均气温通常在20°C至30°C之间，部分地区可能更高。\n- **热带地区**：夏季平均气温较高，通常在25°C至35°C之间，甚至更高。\n- **寒带地区**：夏季平均气温较低，通常在10°C至20°C之间。\n\n### 冬季平均气温\n- **温带地区**：冬季平均气温通常在-10°C至10°C之间，具体取决于纬度。\n- **热带地区**：冬季平均气温较为温和，通常在15°C至25°C之间。\n- **寒带地区**：冬季平均气温极低，通